# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [2]:
# Memuat dataset dari hasil clustering
df = pd.read_csv('https://raw.githubusercontent.com/harisman7/submission-akhir-bmlp/refs/heads/main/result_df.csv')
print(df.head())

   Store  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price         CPI  \
0      1    1643690.90             0        42.31       2.572  211.096358   
1      1    1641957.44             1        38.51       2.548  211.242170   
2      1    1611968.17             0        39.93       2.514  211.289143   
3      1    1409727.59             0        46.63       2.561  211.319643   
4      1    1554806.68             0        46.50       2.625  211.350143   

   Unemployment  day  month  year  Cluster  cluster  
0         8.106    5      2  2010        0        1  
1         8.106   12      2  2010        0        1  
2         8.106   19      2  2010        0        1  
3         8.106   26      2  2010        0        1  
4         8.106    5      3  2010        0        1  


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [3]:
# Memisahkan fitur dan target
X = df.drop('cluster', axis=1)
y = df['cluster']

# Membagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standarisasi fitur
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Menampilkan dimensi data
print(f"Dimensi X_train: {X_train.shape}")
print(f"Dimensi X_test: {X_test.shape}")

Dimensi X_train: (5148, 11)
Dimensi X_test: (1287, 11)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [4]:
# Inisialisasi model
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "SVM": SVC(random_state=42),
    "KNN": KNeighborsClassifier()
}

# Melatih model dan melakukan prediksi
predictions = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    predictions[name] = model.predict(X_test)

# Mengambil hasil prediksi
y_pred_log_reg = predictions["Logistic Regression"]
y_pred_rf = predictions["Random Forest"]
y_pred_dt = predictions["Decision Tree"]
y_pred_svm = predictions["SVM"]
y_pred_knn = predictions["KNN"]

Tulis narasi atau penjelasan algoritma yang Anda gunakan.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [5]:
# Fungsi untuk evaluasi model
def evaluate_model(y_test, y_pred, model_name):
    print(f"\nEvaluasi {model_name} sebelum tuning:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"F1-Score: {f1_score(y_test, y_pred, average='weighted'):.4f}")
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

# Evaluasi masing-masing model
for name, y_pred in zip(
    ["Logistic Regression", "Random Forest", "Decision Tree", "SVM", "K-Nearest Neighbors"],
    [y_pred_log_reg, y_pred_rf, y_pred_dt, y_pred_svm, y_pred_knn]
):
    evaluate_model(y_test, y_pred, name)


Evaluasi Logistic Regression sebelum tuning:
Accuracy: 1.0000
F1-Score: 1.0000
Confusion Matrix:
 [[440   0   0   0]
 [  0 312   0   0]
 [  0   0 332   0]
 [  0   0   0 203]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       440
           1       1.00      1.00      1.00       312
           2       1.00      1.00      1.00       332
           3       1.00      1.00      1.00       203

    accuracy                           1.00      1287
   macro avg       1.00      1.00      1.00      1287
weighted avg       1.00      1.00      1.00      1287


Evaluasi Random Forest sebelum tuning:
Accuracy: 1.0000
F1-Score: 1.0000
Confusion Matrix:
 [[440   0   0   0]
 [  0 312   0   0]
 [  0   0 332   0]
 [  0   0   0 203]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       440
           1       1.00      1.00      1.00       312
           2       1.00      1.00      1.00       332
  

Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [6]:
# Cross-validation setup
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [7]:
# Fungsi untuk tuning model
def tune_model(model, param_grid, X_train, y_train, skf):
    grid_search = GridSearchCV(model, param_grid, cv=skf, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_

# Inisialisasi StratifiedKFold untuk cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Tuning untuk masing-masing model
param_grid_log_reg = {'C': [0.001, 0.01, 0.1, 1, 10], 'penalty': ['l1', 'l2']}
best_log_reg = tune_model(LogisticRegression(random_state=42, max_iter=2000, solver='liblinear'), param_grid_log_reg, X_train, y_train, skf)

param_grid_rf = {'n_estimators': [50, 100, 200], 'max_depth': [10, 20, 30, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
best_rf_clf = tune_model(RandomForestClassifier(random_state=42), param_grid_rf, X_train, y_train, skf)

param_grid_dt = {'max_depth': [10, 20, 30, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
best_dt_clf = tune_model(DecisionTreeClassifier(random_state=42), param_grid_dt, X_train, y_train, skf)

param_grid_svm = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
best_svm_clf = tune_model(SVC(random_state=42), param_grid_svm, X_train, y_train, skf)

param_grid_knn = {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']}
best_knn_clf = tune_model(KNeighborsClassifier(), param_grid_knn, X_train, y_train, skf)

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [8]:
# Fungsi untuk evaluasi model setelah tuning
def evaluate_tuned_model(y_test, best_model, model_name):
    y_pred_best = best_model.predict(X_test)
    print(f"\nEvaluasi {model_name} setelah tuning:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred_best):.4f}")
    print(f"F1-Score: {f1_score(y_test, y_pred_best, average='weighted'):.4f}")
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_best))
    print(classification_report(y_test, y_pred_best))

# Evaluasi model setelah tuning menggunakan loop
models = {
    "Logistic Regression": best_log_reg,
    "Random Forest": best_rf_clf,
    "Decision Tree": best_dt_clf,
    "SVM": best_svm_clf,
    "K-Nearest Neighbors": best_knn_clf
}

for model_name, best_model in models.items():
    evaluate_tuned_model(y_test, best_model, model_name)


Evaluasi Logistic Regression setelah tuning:
Accuracy: 1.0000
F1-Score: 1.0000
Confusion Matrix:
 [[440   0   0   0]
 [  0 312   0   0]
 [  0   0 332   0]
 [  0   0   0 203]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       440
           1       1.00      1.00      1.00       312
           2       1.00      1.00      1.00       332
           3       1.00      1.00      1.00       203

    accuracy                           1.00      1287
   macro avg       1.00      1.00      1.00      1287
weighted avg       1.00      1.00      1.00      1287


Evaluasi Random Forest setelah tuning:
Accuracy: 1.0000
F1-Score: 1.0000
Confusion Matrix:
 [[440   0   0   0]
 [  0 312   0   0]
 [  0   0 332   0]
 [  0   0   0 203]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       440
           1       1.00      1.00      1.00       312
           2       1.00      1.00      1.00       332
  

In [9]:
# Fungsi untuk cross-validation performance
def cross_val_performance(model, X, y, skf):
    cross_val_accuracy = cross_val_score(model, X, y, cv=skf, scoring='accuracy')
    cross_val_f1 = cross_val_score(model, X, y, cv=skf, scoring='f1_weighted')
    return cross_val_accuracy, cross_val_f1

# Menyusun model dan nama model dalam dictionary
models = {
    "Logistic Regression": best_log_reg,
    "Random Forest": best_rf_clf,
    "Decision Tree": best_dt_clf,
    "SVM": best_svm_clf,
    "K-Nearest Neighbors": best_knn_clf
}

# Cross-validation untuk masing-masing model
for name, model in models.items():
    cross_val_accuracy, cross_val_f1 = cross_val_performance(model, X, y, skf)
    print(f"\nCross-validation untuk {name}:")
    print(f"Accuracy: {cross_val_accuracy.mean():.4f} ± {cross_val_accuracy.std():.4f}")
    print(f"F1-Score: {cross_val_f1.mean():.4f} ± {cross_val_f1.std():.4f}")


Cross-validation untuk Logistic Regression:
Accuracy: 0.9992 ± 0.0009
F1-Score: 0.9992 ± 0.0009

Cross-validation untuk Random Forest:
Accuracy: 0.9997 ± 0.0004
F1-Score: 0.9997 ± 0.0004

Cross-validation untuk Decision Tree:
Accuracy: 0.9998 ± 0.0003
F1-Score: 0.9998 ± 0.0003

Cross-validation untuk SVM:
Accuracy: 0.9998 ± 0.0003
F1-Score: 0.9998 ± 0.0003

Cross-validation untuk K-Nearest Neighbors:
Accuracy: 0.9998 ± 0.0003
F1-Score: 0.9998 ± 0.0003


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

Berdasarkan hasil evaluasi model (Logistic Regression, Random Forest, Decision Tree, dan SVM), berikut adalah analisis dan rekomendasi tindakan lanjutan:
#### 1. Perbandingan Hasil Evaluasi
- **Logistic Regression**: Akurasi 0.9992, F1-Score 0.9992
- **Random Forest**: Akurasi 0.9997, F1-Score 0.9997
- **Decision Tree**: Akurasi 0.9998, F1-Score 0.9998
- **SVM**: Akurasi 0.9998, F1-Score 0.9998

Dari hasil ini, **Decision Tree** dan **SVM** memberikan hasil yang paling tinggi dengan akurasi dan F1-score sebesar 0.9998, yang menunjukkan bahwa keduanya bekerja sangat baik. **Random Forest** juga memberikan hasil yang sangat baik dengan akurasi 0.9997 dan F1-score 0.9997, sedangkan **Logistic Regression** memberikan hasil yang sedikit lebih rendah, tetapi tetap sangat baik dengan akurasi 0.9992 dan F1-score 0.9992.

#### 2. Identifikasi Kelemahan Model
- ### **Precision dan Recall**  
Meskipun akurasi dan F1-Score sangat tinggi pada semua model, masalah **precision** atau **recall** rendah untuk kelas tertentu mungkin terjadi, terutama jika dataset tidak seimbang. **Logistic Regression**  meskipun menunjukkan hasil yang baik dengan akurasi 0.9992 dan F1-Score 0.9992, bisa mengalami masalah pada kelas minoritas, meskipun hal ini belum dapat dipastikan tanpa analisis lebih lanjut. Pada **Random Forest**, meskipun akurasi tinggi dengan nilai 0.9997 dan F1-Score 0.9997, model bisa lebih fokus pada kelas mayoritas, menyebabkan **precision** atau **recall** rendah pada kelas minoritas. **Decision Tree** juga berpotensi mengalami hal serupa jika dataset tidak seimbang, meskipun hasil secara keseluruhan sangat baik dengan akurasi 0.9998 dan F1-Score 0.9998. Begitu pula dengan **SVM**, yang meskipun menunjukkan hasil yang sangat baik dengan akurasi 0.9998 dan F1-Score 0.9998, dapat mengalami masalah serupa jika distribusi kelas tidak merata.

- ### **Overfitting atau Underfitting**  
Secara umum, model-model yang diuji menunjukkan hasil yang sangat tinggi, yang menandakan kemungkinan besar tidak ada **underfitting**. Namun, beberapa model, seperti **Random Forest**, **Decision Tree**, dan **SVM**, berisiko mengalami **overfitting** karena kompleksitasnya. **Random Forest** dan **Decision Tree**, yang cenderung sangat fleksibel, berpotensi menyesuaikan diri terlalu banyak dengan data pelatihan dan kurang menggeneralisasi pada data uji. **SVM**, meskipun kinerjanya sangat baik, juga bisa mengalami **overfitting** jika parameter tidak disesuaikan dengan baik.

#### 3. Rekomendasi Tindakan Lanjutan
- ### **1. Analisis Confusion Matrix**  
Confusion matrix penting untuk memahami distribusi prediksi model, khususnya kesalahan tipe I (false positives) dan tipe II (false negatives). Analisis ini membantu menentukan prioritas antara Precision dan Recall sesuai kebutuhan aplikasi, misalnya mengurangi false negatives untuk aplikasi yang memerlukan sensitivitas tinggi.

- ### **2. Validasi dengan Data Baru**  
Validasi menggunakan **k-fold cross-validation** dan dataset terpisah (test set) memastikan performa model tidak hanya baik di data pelatihan tetapi juga dapat diandalkan di dunia nyata. Langkah ini penting untuk mengidentifikasi kemampuan generalisasi model.

- ### **3. Pengurangan Overfitting**  
Overfitting pada Decision Tree dapat diatasi dengan pruning, sedangkan Random Forest dapat dioptimalkan melalui pengaturan jumlah pohon (**n_estimators**) dan kedalaman maksimum pohon (**max_depth**). Langkah ini memastikan model tetap efisien dan tidak hanya menghafal data.

- ### **4. Hyperparameter Tuning**  
Menggunakan **Grid Search** atau **Random Search** untuk optimasi hyperparameter seperti **C** dan **gamma** pada SVM, atau kedalaman pohon pada Random Forest, membantu meningkatkan performa model. Tuning ini memastikan model dapat menangani data kompleks dengan lebih baik.